# Object Detection
## Finding the Pentagon Figure on the page
### Using Python

Creating and Training a model to find the Pentagon figure on the page of the MMSE Document


> Uses methods and modules with the SKlearn and SKImage libraries in python 
>
> --- [scikit-learn 0.19.1. (2017, October). Simple and efficient tools for data mining and data analysis](http://scikit-learn.org/stable/index.html)

In [ ]:
# imports
# Import the functions to calculate feature descriptors
from skimage.feature import local_binary_pattern
from skimage.feature import hog
from skimage.io import imread
from sklearn.externals import joblib
import cv2
# To read file names
import glob
import os
import numpy as np

Must create a correct file structure to save the features for training method

In [ ]:
# file structure for feature ies
path = r'C:\Users\KinectProcessing\Documents\DigiPen Studies\training_mmse_pentagons' #change to the base dir for files
pos_im_path = os.path.join(path, 'pos_pent')
neg_im_path = os.path.join(path, 'neg_pent')
pos_feat_ph = os.path.join(path, "pos_feat_ph")
neg_feat_ph = os.path.join(path, "neg_feat_ph")

In using the HOG detection paramaters must be set


Parameters:	
image : (M, N[, C]) ndarray
    Input image.

orientations : int, optional
    Number of orientation bins.

pixels_per_cell : 2-tuple (int, int), optional
    Size (in pixels) of a cell.

cells_per_block : 2-tuple (int, int), optional
    Number of cells in each block.

block_norm : str {‘L1’, ‘L1-sqrt’, ‘L2’, ‘L2-Hys’}, optional
    Block normalization method:

In [ ]:
#parameters for HOG
#reference: http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)
des_type = "HOG"

Next create the directory to save the features in

In [ ]:
 # If feature directories don't exist, create them
if not os.path.isdir(pos_feat_ph):
    os.makedirs(pos_feat_ph)

# If feature directories don't exist, create them
if not os.path.isdir(neg_feat_ph):
    os.makedirs(neg_feat_ph)

Positive image feature input for the MMSE pentagon example

In [ ]:
from IPython.display import Image
Image(filename=r'C:\Users\KinectProcessing\Documents\DigiPen Studies\training_mmse_pentagons\pos_pent\EXTRACTED_PENT-00009121  01-31-13  FU3img-9.png') 

Negative image feature input for MMSE example

In [ ]:
from IPython.display import Image
Image(filename=r'C:\Users\KinectProcessing\Documents\DigiPen Studies\training_mmse_pentagons\pos_pent\EXTRACTED_PENT-00987978  FU4  10-21-09img-11.png') 

For each image path in the positive directory we first read it into the im variable and convert it to graysale
Then it is resized to 128x128 image
Finally create the HOG descriptor for the image. A 1D (flattened) array is returned.

In [ ]:
print("Calculating the descriptors for the positive samples and saving them")
for im_path in glob.glob(os.path.join(pos_im_path, "*")):
    im = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE) #Reads image in and convers to grayscale
    im = cv2.resize(im, (128,128))
    if des_type == "HOG":
        fd, _ = hog(im, orientations, pixels_per_cell, cells_per_block, visualise = True, transform_sqrt=True)
    print(fd.shape)
    fd_name = os.path.split(im_path)[1].split(".")[0] + ".feat"
    fd_path = os.path.join(pos_feat_ph, fd_name)
    joblib.dump(fd, fd_path) #dumps feature detection in the save path
print("Positive features saved in {}".format(pos_feat_ph))

In [ ]:
print("Calculating the descriptors for the negative samples and saving them")
for im_path in glob.glob(os.path.join(neg_im_path, "*")):
    im = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (128,128))
    if des_type == "HOG":
        fd, _ = hog(im,  orientations, pixels_per_cell, cells_per_block, visualise = True, transform_sqrt=True)
    fd_name = os.path.split(im_path)[1].split(".")[0] + ".feat"
    fd_path = os.path.join(neg_feat_ph, fd_name)
    joblib.dump(fd, fd_path)
print("Negative features saved in {}".format(neg_feat_ph))

In [ ]:
print("Completed calculating features from training images")

# Train Classifier
## Training with linear SVM


In [ ]:
# Import the required modules
from skimage.feature import local_binary_pattern
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.decomposition import PCA
import glob
import os
import pandas as pd
import numpy as np

path = r'\training_mmse_pentagons' # change to dir of testing path
pos_im_path = os.path.join(path, 'pos_pent')
neg_im_path = os.path.join(path, 'neg_pent')

pos_feat_ph = os.path.join(path, "pos_feat_ph")
neg_feat_ph = os.path.join(path, "neg_feat_ph")

model_path =  os.path.join(path, "models", "svm.model") # where to save the model

pos_feat_path =  pos_feat_ph
neg_feat_path =  neg_feat_ph

# Classifiers supported
clf_type = "LIN_SVM"

fds = []
labels = []
# Load the positive features
for feat_path in glob.glob(os.path.join(pos_feat_path,"*.feat")):
    fd = joblib.load(feat_path)
    fds.append(fd)
    labels.append(1)

# Load the negative features
for feat_path in glob.glob(os.path.join(neg_feat_path,"*.feat")):
    fd = joblib.load(feat_path)
    fds.append(fd)
    labels.append(0)

if clf_type is "LIN_SVM":
    clf = LinearSVC()
    print("Training a Linear SVM Classifier")
    print(len(fds), len(labels))
    clf.fit(fds, labels)
    # If feature directories don't exist, create them
    if not os.path.isdir(os.path.split(model_path)[0]):
        os.makedirs(os.path.split(model_path)[0])
    joblib.dump(clf, model_path)
    print("Classifier saved to {}".format(model_path))
